In [1]:
import pandas as pd
from sqlalchemy import create_engine

from config import (user, password, port, address, database)


# EXTRACT

### We have three data sources that we are extracting from:
    - Fuel Stations: https://afdc.energy.gov/stations/#/analyze?fuel=ELEC&ev_levels=all 
    - Electric Vehicle registrations by state: https://afdc.energy.gov/data/10962
    - States and abbreviations: https://www.50states.com/abbreviations.htm    

### Process:
    Fuel Stations was a csv files found on above source site.
    Electric Vehicle registrations by state was a csv files found on above source site.
    States and abbreviations were pulled from a table on the aforementioned website. It was copied into an Excel file for use.
    
    All three csv files were extracted into their own pandas dataframe.
    
    We used 2 out of 64 columns from the Fuel Stations csv; State & Fuel Type Code.
    
    We used 2 out of 2 columns from the Electric Vehicle registrations by state csv; State and Registration.  
    
    We used 2 out of 2 columns from the States and abbreviations csv file that we created; US State and Abbreviation.
    
    
    

### Store Fuel Stations CSV into Dataframe

In [2]:
csv_file = "Resources/Fuel Stations all Fuel Types USA.csv"
fuel_stations_df = pd.read_csv(csv_file,low_memory=False)
fuel_stations_df.head()

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source
0,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN
1,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN
2,CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,NaN,Fort Smith,AR,72903,NaN,479-783-3188,E,...,NaN,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN
3,CNG,Clean Energy - Logan International Airport,1000 Cottage St Ext,"From Route 1, take the first exit after Callah...",East Boston,MA,2128,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN
4,CNG,Clean Energy - Everett - National Grid,16 Rover St,"Rt 16, exit to Rt 99, to Dexter St to Rover. O...",Everett,MA,2149,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,NaN,Q,3000 3600,HD,NaN,NaN


### Create new data Frame with columns

In [3]:
station_cols = [ "State","Fuel Type Code"]
new_fuel_stations_df = fuel_stations_df[station_cols].copy()
new_fuel_stations_df.head()

,State,Fuel Type Code
0,GA,CNG
1,TX,CNG
2,AR,CNG
3,MA,CNG
4,MA,CNG


# TRANSFORM

#### Fuel Stations dataframe transformations: 
        1. Rename columns 
        2. Selected for Fuel Type of Electric only
        3. Set the index for the primary key 
        

#### State Registrations dataframe:
        1. Changed column header names

#### State Abbreviations dataframe:
        1. Changed column header names

### Transform Fuel Stations Data Frame

In [4]:
# Rename Column Headers & Clean up Duplicates

new_fuel_stations_df = new_fuel_stations_df.rename(columns={"State": "state_abbr",
                                                           "Fuel Type Code": "fuel_type"})

new_fuel_stations_df.head()

,state_abbr,fuel_type
0,GA,CNG
1,TX,CNG
2,AR,CNG
3,MA,CNG
4,MA,CNG


In [5]:
# Select all electric fuel types 
new_fuel_stations_df = new_fuel_stations_df[new_fuel_stations_df["fuel_type"]=="ELEC"]

new_fuel_stations_df.count()

state_abbr    47086
fuel_type     47086
dtype: int64

In [6]:
new_fuel_stations_df = new_fuel_stations_df.reset_index(drop=True)
new_fuel_stations_df.head()

,state_abbr,fuel_type
0,CA,ELEC
1,CA,ELEC
2,CA,ELEC
3,CA,ELEC
4,CA,ELEC


### Transform Registrations Data Frame

In [7]:
csv_file = "Resources/10962_ev_registration_counts_by_state.csv"
registration_df = pd.read_csv(csv_file,low_memory=False)
registration_df.dtypes

State                 object
Registration Count    object
dtype: object

In [8]:
registration_cols = ["State","Registration Count"]
new_registration_df= registration_df[registration_cols].copy()
new_registration_df.head()

,State,Registration Count
0,Alabama,"1,450"
1,Alaska,530
2,Arizona,"15,000"
3,Arkansas,520
4,California,"256,800"


In [9]:
new_registration_df = new_registration_df.rename(columns={"State":"state",
                                                         "Registration Count":"registrations"})
new_registration_df.head()

,state,registrations
0,Alabama,"1,450"
1,Alaska,530
2,Arizona,"15,000"
3,Arkansas,520
4,California,"256,800"


### Transform State Abbreviations Data Frame

In [10]:
csv_file = "Resources/states.csv"
states_df= pd.read_csv(csv_file, low_memory=False)
states_df.head()

,US STATE,ABBREVIATION
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [11]:
new_state_df = states_df.rename(columns={"US STATE":"state",
                                           "ABBREVIATION":"state_abbr"})
new_state_df.head()

,state,state_abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


# LOAD

### Steps
1. Create database connection 
2. Load DataFrames into database

 ### Create Database Connection

In [12]:
engine = create_engine(f'postgresql://{user}:{password}@{address}:{port}/{database}')

conn = engine.connect()

In [13]:
engine.table_names()

['state_abbreviations', 'registrations', 'fuel_stations']

In [14]:
new_fuel_stations_df.to_sql(name='fuel_stations', con = engine, if_exists='append', index=True)

In [15]:
new_registration_df.to_sql(name='registrations', con = engine, if_exists='append', index=True)

In [16]:
new_state_df.to_sql(name='state_abbreviations', con = engine, if_exists='append', index=True)